In [27]:
import tensorflow as tf
import numpy as np
import pickle

with open('../data/conv_lstm_sequence_data.pc', 'rb') as f:
    raw_data = pickle.load(f)
raw_data = np.expand_dims(raw_data, axis=-1)
print("raw dataset shape: {}".format(raw_data.shape))

raw dataset shape: (17006, 30, 40, 1)


In [28]:
### THESE CODES ARE MEANT FOR TF2

In [29]:
train, test = raw_data[:15000], raw_data[15000:]
print("train, test shapes: {}-{}".format(train.shape, test.shape))

train, test shapes: (15000, 30, 40, 1)-(2006, 30, 40, 1)


In [ ]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 10

train_ds = Dataset.from_tensor_slices(train)
test_ds = Dataset.from_tensor_slices(test)

test_seq = train_ds.batch(seq_len+1, drop_remainder=True)
sequences = train_ds.batch(seq_len+1, drop_remainder=True)

test_ds = test_seq.map(lambda x: (x[:-1], x[1:]))
test_ds = test_ds.batch(batch_size, drop_remainder=True)
ds = sequences.map(lambda x: (x[:-1], x[1:]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

In [ ]:
ds

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', input_shape=(None, 30, 40, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(filters=1, kernel_size=(3,3,3), activation='sigmoid', padding='same')
])

model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
model.fit(ds, epochs=5, validation_data=test_ds)

In [26]:
model.evaluate(test_ds)

78/78 [==============================] - 52s 661ms/step - loss: 0.3015


0.3014840982281245

# COMPARE

in this section I compare it with return_sequence=False

In [30]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 10

train_ds = Dataset.from_tensor_slices(train)
test_ds = Dataset.from_tensor_slices(test)

test_seq = train_ds.batch(seq_len+1, drop_remainder=True)
sequences = train_ds.batch(seq_len+1, drop_remainder=True)

test_ds = test_seq.map(lambda x: (x[:-1], x[-1]))
test_ds = test_ds.batch(batch_size, drop_remainder=True)
ds = sequences.map(lambda x: (x[:-1], x[-1]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

In [42]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', input_shape=(None, 30, 40, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), padding='same')
])

model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_31 (ConvLSTM2D) (None, None, 30, 40, 20)  15200     
_________________________________________________________________
batch_normalization_30 (Batc (None, None, 30, 40, 20)  80        
_________________________________________________________________
conv_lst_m2d_32 (ConvLSTM2D) (None, 30, 40, 20)        28880     
_________________________________________________________________
batch_normalization_31 (Batc (None, 30, 40, 20)        80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 40, 1)         181       
Total params: 44,421
Trainable params: 44,341
Non-trainable params: 80
_________________________________________________________________


In [ ]:
model.fit(ds, epochs=5, validation_data=test_ds)

Epoch 1/5
78/78 [==============================] - 179s 2s/step - loss: 0.3572 - val_loss: 0.4628
Epoch 2/5
78/78 [==============================] - 176s 2s/step - loss: 0.2688 - val_loss: 0.4170
Epoch 3/5
78/78 [==============================] - 184s 2s/step - loss: 0.2645 - val_loss: 0.3651
Epoch 4/5
39/78 [==============>...............] - ETA: 1:07 - loss: 0.2755

In [ ]:
### THESE CODES ARE MEANT FOR TF1.14

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       input_shape=(None, 30, 40, 1),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                   activation='sigmoid',
                   padding='same', data_format='channels_last'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta')

model.summary()

In [ ]:
print(raw_data.shape)
seq_len = 18
bs = 10

remainder = raw_data.shape[0]%seq_len
raw_data = raw_data[:raw_data.shape[0]-remainder,:]

print(raw_data.shape)

seq = np.array(np.split(raw_data, raw_data.shape[0]//seq_len))

In [ ]:
input_data, target = seq[:,:-1, :], seq[:, 1:, :]
print(input_data.shape, target.shape)


In [ ]:
model.fit(input_data, target, batch_size=10, epochs=10, validation_split=0.05)

In [ ]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 5

ds = Dataset.from_tensor_slices(raw_data)
sequences = ds.batch(seq_len+1, drop_remainder=True).shuffle(10000).batch(batch_size, drop_remainder=True)

ds = sequences.map(lambda x: x[:,:-1])
target = sequences.map(lambda x: x[:,1:])


In [ ]:
ds, target

In [ ]:
seq.fit(ds, target, steps_per_epoch=10)